In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_MODEL_NAME'] = 'gpt-3.5-turbo'
os.environ['SERPER_API_KEY'] = os.getenv('SERPER_API_KEY')

In [5]:
from crewai import Agent, Task, Crew

crewAI Tools

In [3]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

Creating Agents

In [6]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [9]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

Creating Tasks

In [10]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [11]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

In [14]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [15]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

In [16]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [17]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the work of continuously monitoring and analyzing market data for AAPL to the Data Analyst.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL", "context": "The Data Analyst is responsible for using statistical modeling and machine learning to identify trends and predict market movements for AAPL. They need to provide insights and alerts about significant market opportunities or threats."}

> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL to identify trends and predict market movements.

Action: Read website content
Action Input: {"website_url": "https://finance.yah

In [18]:
from IPython.display import Markdown
Markdown(result)

**Risk Analysis Report for AAPL Trading Strategies:**

**Long-Term Investment Strategy:**
The long-term investment strategy for AAPL involves purchasing shares at regular intervals over an extended period. This strategy aims to benefit from the company's growth potential and stable financial performance. However, potential risks associated with this strategy include market volatility, economic downturns, and changes in industry trends. To mitigate these risks, investors should consider diversifying their portfolio across different asset classes and conducting thorough research before making investment decisions.

**Options Trading Strategy:**
The options trading strategy for AAPL involves implementing covered calls to generate additional income while limiting downside risk. Risks associated with this strategy include fluctuations in option prices, market uncertainty, and unexpected company developments. To address these risks, traders should closely monitor market conditions, set clear risk management guidelines, and adjust their positions accordingly to protect against potential losses.

**Technical Analysis Strategy:**
The technical analysis strategy for AAPL focuses on monitoring key technical indicators to identify optimal entry and exit points. Risks associated with this strategy include false signals, market manipulation, and unexpected news events. Traders should use a combination of technical analysis tools, conduct thorough research, and stay informed about market trends to make informed trading decisions and minimize risks.

**Diversification Strategy:**
The diversification strategy for AAPL involves adding other tech stocks or ETFs to the portfolio to spread out risk and potentially enhance returns. Risks associated with this strategy include sector-specific downturns, correlation between assets, and geopolitical events. To mitigate these risks, investors should diversify across different sectors, conduct regular portfolio reviews, and adjust their allocations based on market conditions to maintain a balanced risk profile.

In conclusion, by carefully analyzing the risks associated with the proposed trading strategies for AAPL and implementing appropriate mitigation strategies, investors and traders can optimize their trading performance and achieve their financial goals in line with current market conditions and risk tolerance levels.